# Azure Machine Learning Pipeline with AutoMLStep
This notebook implements AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
How to use AzureML Dataset to load data for AutoML via AML Pipeline. You first need to have an **Azure Machine Learning Notebook VM**, before moving on to the next steps.

Here are the different steps :
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

banking-ws
alexeirg
francecentral
af783b88-9530-433e-9520-32a8accf75a5


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,banking-ws,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run.

**Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Create from scatch.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Create from scatch.
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bankmarketing-ds"
description_text = "Bank Marketing DataSet"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
    dataset = Dataset.get_by_name(ws, name=key)
    df = dataset.to_pandas_dataframe()
    
    # Create AML Dataset and register it into Workspace
#     example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
#     dataset = Dataset.Tabular.from_delimited_files(example_data)        
#     #Register Dataset in Workspace
#     dataset = dataset.register(workspace=ws,
#                                name=key,
#                                description=description_text)

In [7]:
df = dataset.to_pandas_dataframe()
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [ ]:
df.describe()

### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [8]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [10]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [11]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [12]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [13]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [f6e5a807][8b570895-48fc-4db5-81cf-92ab9496c4aa], (This step will run and generate new outputs)
Submitted PipelineRun 37fca20c-745c-4d99-aae4-3aff0e2f04b3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/37fca20c-745c-4d99-aae4-3aff0e2f04b3?wsid=/subscriptions/af783b88-9530-433e-9520-32a8accf75a5/resourcegroups/alexeirg/workspaces/banking-ws&tid=5b38c313-3bf3-4f5b-90e6-8e32480e8986


In [14]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [15]:
pipeline_run.wait_for_completion()

PipelineRunId: 37fca20c-745c-4d99-aae4-3aff0e2f04b3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/37fca20c-745c-4d99-aae4-3aff0e2f04b3?wsid=/subscriptions/af783b88-9530-433e-9520-32a8accf75a5/resourcegroups/alexeirg/workspaces/banking-ws&tid=5b38c313-3bf3-4f5b-90e6-8e32480e8986
PipelineRun Status: Running


StepRunId: 3b2d5412-f00a-4604-946b-2c49f41eb697
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/3b2d5412-f00a-4604-946b-2c49f41eb697?wsid=/subscriptions/af783b88-9530-433e-9520-32a8accf75a5/resourcegroups/alexeirg/workspaces/banking-ws&tid=5b38c313-3bf3-4f5b-90e6-8e32480e8986
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.
{'runId': '3b2d5412-f00a-4604-946b-2c

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. We will examine the outputs by retrieve output data and running some tests.

In [16]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/3b2d5412-f00a-4604-946b-2c49f41eb697/metrics_data, 1 files out of an estimated total of 1


In [17]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,3b2d5412-f00a-4604-946b-2c49f41eb697_54,3b2d5412-f00a-4604-946b-2c49f41eb697_50,3b2d5412-f00a-4604-946b-2c49f41eb697_58,3b2d5412-f00a-4604-946b-2c49f41eb697_0,3b2d5412-f00a-4604-946b-2c49f41eb697_9,3b2d5412-f00a-4604-946b-2c49f41eb697_29,3b2d5412-f00a-4604-946b-2c49f41eb697_28,3b2d5412-f00a-4604-946b-2c49f41eb697_52,3b2d5412-f00a-4604-946b-2c49f41eb697_49,3b2d5412-f00a-4604-946b-2c49f41eb697_45,...,3b2d5412-f00a-4604-946b-2c49f41eb697_23,3b2d5412-f00a-4604-946b-2c49f41eb697_51,3b2d5412-f00a-4604-946b-2c49f41eb697_26,3b2d5412-f00a-4604-946b-2c49f41eb697_7,3b2d5412-f00a-4604-946b-2c49f41eb697_1,3b2d5412-f00a-4604-946b-2c49f41eb697_15,3b2d5412-f00a-4604-946b-2c49f41eb697_5,3b2d5412-f00a-4604-946b-2c49f41eb697_8,3b2d5412-f00a-4604-946b-2c49f41eb697_11,3b2d5412-f00a-4604-946b-2c49f41eb697_59
precision_score_weighted,[0.905693502555877],[0.9000518141698153],[0.9077209216378826],[0.9051980543721705],[0.8769191713835482],[0.9022929732142668],[0.903873789890378],[0.9085610772055459],[0.8996799705499371],[0.788565560086672],...,[0.9162722759183641],[0.9092432341499307],[0.9021382069947883],[0.9005211086889047],[0.9067326786550023],[0.8973958505740813],[0.8973758906640772],[0.9021040535657128],[0.8798312026519207],[0.906281456407151]
AUC_weighted,[0.941420902589067],[0.9382834395671366],[0.9471748476883264],[0.9446537630106308],[0.8367454111998399],[0.9374697831052131],[0.9395652842379415],[0.9403608800271188],[0.9406868983248958],[0.5],...,[0.9145142049506619],[0.9434728728695352],[0.9394485845063509],[0.9290011799639528],[0.942328104073932],[0.9439716252938334],[0.9388252597495217],[0.941632999720291],[0.8255209346351837],[0.9466441417660929]
average_precision_score_macro,[0.8117322160748517],[0.8088396395612727],[0.8232223868471997],[0.8171134643523112],[0.7111293501560303],[0.8113281814330582],[0.8032487617207208],[0.8046021931218315],[0.8045325717653632],[0.5],...,[0.7566249770652664],[0.8130250782253756],[0.8028257572649862],[0.8044713126721892],[0.8028233128807174],[0.8180629123186703],[0.8006349647033797],[0.8161730247772492],[0.7213401897274458],[0.8232923165811681]
precision_score_macro,[0.7841169254587081],[0.7740509789925003],[0.8023935583498787],[0.7781441711329087],[0.6013217264838592],[0.8093132277582819],[0.784265962348154],[0.785334883295727],[0.7667567309797394],[0.4440060698027314],...,[0.646011466298251],[0.7911521311039829],[0.782781125204717],[0.8065451980757572],[0.7826005666081467],[0.8515415609080743],[0.7856451172940535],[0.7845613466137928],[0.6155422041262002],[0.794377847854753]
matthews_correlation,[0.522998482568272],[0.49318228471542264],[0.5286652210598994],[0.5216286298277554],[0.30391853590071993],[0.48864489691810753],[0.5123580976029641],[0.5388381690877376],[0.4931955844544311],[0.0],...,[0.4455247597901918],[0.5414091775476605],[0.5025170151065972],[0.47805766735773875],[0.52924365161229],[0.39340196123415067],[0.47077965319103915],[0.5016093372462171],[0.3340780765517017],[0.5231250045786537]
recall_score_weighted,[0.9116843702579667],[0.9077389984825494],[0.9156297420333839],[0.9101669195751139],[0.7165402124430956],[0.9132018209408195],[0.9110773899848255],[0.9128983308042489],[0.9059180576631259],[0.8880121396054628],...,[0.7408194233687405],[0.9144157814871017],[0.9101669195751139],[0.9119878603945372],[0.9116843702579667],[0.9068285280728376],[0.908649468892261],[0.9104704097116844],[0.7553869499241275],[0.9138088012139606]
average_precision_score_weighted,[0.9520081448351643],[0.9510163307479341],[0.9551769074921843],[0.9535822261506017],[0.915021980050871],[0.9514215544812085],[0.9499340331794855],[0.9502984240004876],[0.9503230429639213],[0.8011068409624182],...,[0.9366995880319352],[0.9524875074933626],[0.9497977778114575],[0.9480633299225389],[0.950146970738567],[0.9536793029588512],[0.9492172134275333],[0.9529980385601647],[0.9138998635998103],[0.9551472630359654]
f1_score_macro,[0.7598459659998121],[0.7440645919566662],[0.7601198968422067],[0.7598074755896701],[0.5934600747999751],[0.7

### Retrieve the Best Model

In [18]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/3b2d5412-f00a-4604-946b-2c49f41eb697/model_data, 1 files out of an estimated total of 1


In [19]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/m...
                                                  PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('39', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.3, gamma=5, max_depth=6, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.7291666666666667, reg_lambda=1

In [20]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('39', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.3, gamma=5, max_depth=6, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.7291666666666667, reg_lambda=1.5625, subsample=0.8, tree_method='auto'))], verbose=False)), ('32', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False))

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [21]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [22]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [23]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,3531,105
1,238,246


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [24]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,71f95af5-9b7f-414b-a0cb-e894a4ae8fb3,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [25]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [26]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [27]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  c5f1284c-4ffc-46ae-aace-8080a5cf5e26


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [28]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …